In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class TemperatureScaling(nn.Module):
    def __init__(self, temperature=1.0):
        super().__init__()
        self.temperature = nn.Parameter(torch.ones(1) * temperature)
        
    def forward(self, logits):
        return logits / self.temperature

def find_optimal_temperature(model, val_loader, device):
    # 모델의 출력을 저장할 리스트
    logits_list = []
    labels_list = []
    
    # 검증 세트에 대한 모델의 출력(로짓) 수집
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            logits = model(inputs)
            logits_list.append(logits.cpu())
            labels_list.append(labels)
    
    logits = torch.cat(logits_list).to(device)
    labels = torch.cat(labels_list).to(device)
    
    # Temperature Scaling 모델 초기화
    temperature_model = TemperatureScaling().to(device)
    
    # NLL 손실 함수와 옵티마이저 정의
    nll_criterion = nn.CrossEntropyLoss()
    optimizer = optim.LBFGS([temperature_model.temperature], lr=0.01, max_iter=50)
    
    # 최적화 함수 정의
    def eval():
        optimizer.zero_grad()
        scaled_logits = temperature_model(logits)
        loss = nll_criterion(scaled_logits, labels)
        loss.backward()
        return loss
    
    # 최적화 수행
    optimizer.step(eval)
    
    # 최적의 temperature 반환
    return temperature_model.temperature.item()

# 사용 예시
optimal_temperature = find_optimal_temperature(model, val_loader, device)
print(f"Optimal Temperature: {optimal_temperature}")

# 최적의 temperature를 적용한 TemperatureScaling 모델 생성
calibrated_model = nn.Sequential(
    model,
    TemperatureScaling(optimal_temperature)
)